## Mizzou vs Indiana State History

In [39]:
from urllib.request import urlopen
import pandas as pd

In [40]:
from bs4 import BeautifulSoup

In [142]:
years = []
for year in range(2009,2020):
    years.append(str(year))

In [143]:
base_schedule_url = 'https://gosycamores.com/sports/womens-soccer/schedule/'
schedule_urls = []
for year in years:
    isu_schedule_url = base_schedule_url+year
    schedule_urls.append(isu_schedule_url)

In [144]:
print(schedule_urls)

['https://gosycamores.com/sports/womens-soccer/schedule/2009', 'https://gosycamores.com/sports/womens-soccer/schedule/2010', 'https://gosycamores.com/sports/womens-soccer/schedule/2011', 'https://gosycamores.com/sports/womens-soccer/schedule/2012', 'https://gosycamores.com/sports/womens-soccer/schedule/2013', 'https://gosycamores.com/sports/womens-soccer/schedule/2014', 'https://gosycamores.com/sports/womens-soccer/schedule/2015', 'https://gosycamores.com/sports/womens-soccer/schedule/2016', 'https://gosycamores.com/sports/womens-soccer/schedule/2017', 'https://gosycamores.com/sports/womens-soccer/schedule/2018', 'https://gosycamores.com/sports/womens-soccer/schedule/2019']


In [145]:
isu_hist_schedule = pd.DataFrame(index=None)
opponent_list = []
date_list = []
result_list = []    
year_list = []
for schedule_url in schedule_urls:
    # empty lists

    
    # Set up parser
    page = urlopen(schedule_url)
    html = page.read().decode('utf-8')
    soup = BeautifulSoup(html,"html.parser")
    
    # pull info
    otags = soup.find_all('div',{"class":'sidearm-schedule-game-opponent-text'})
    dtags = soup.find_all('div',{"class":'sidearm-schedule-game-opponent-date'})
    rtags = soup.find_all('div',{'class':'sidearm-schedule-game-result'})
    
    # get result tag
    for rtag in rtags[1::2]:
        game_result = rtag.find('span')
        if game_result:
            game_results = rtag.text
            
            #store names
            result_list.append(game_results.strip().replace('\n'," "))
            # print(game_results.strip().replace('\n'," ")+", year: "+schedule_url[-4:])
            
    # get opponent tag
    for otag in otags[1::2]:

        if opponent:
            opponents = otag.text

            # store names
            opponent_list.append(opponents.strip().replace("\n"," "))

            # print(opponent_names.strip())
        year_list.append(schedule_url[-4:])
    # get date tag
    for dtag in dtags:
        game_date = dtag.find('span')
        if game_date:
            game_dates = dtag.text
            
            # store names
            date_list.append(game_dates.strip().replace("\n"," "))
            # print(game_dates.strip())


In [146]:
len(result_list)

220

In [147]:
isu_hist_schedule['opponent'] = opponent_list
isu_hist_schedule['game_full_date'] = date_list
isu_hist_schedule['game_year'] = year_list
isu_hist_schedule['game_result'] = result_list

isu_hist_schedule[['game_date','game_full_date']] = isu_hist_schedule['game_full_date'].str.split("(",expand=True)
isu_hist_schedule[['game_day_of_week','game_time']] = isu_hist_schedule['game_full_date'].str.split(")",expand=True)
isu_hist_schedule[['game_result','game_score']] = isu_hist_schedule['game_result'].str.split(',',expand=True)

In [166]:
isu_hist_schedule = isu_hist_schedule.replace('OT W',"W")
isu_hist_schedule = isu_hist_schedule.replace('OT L',"L")
isu_hist_schedule = isu_hist_schedule.replace('2OT W',"W")
isu_hist_schedule = isu_hist_schedule.replace('2OT L',"L")
isu_hist_schedule = isu_hist_schedule.replace('4-3 PKs T',"T")
isu_hist_schedule = isu_hist_schedule.replace("W(3-2 in PK's) T","T")
isu_hist_schedule = isu_hist_schedule.replace("","N/A")

In [167]:
isu_hist_schedule['game_result'].value_counts()

L      118
W       70
T       19
N/A     13
Name: game_result, dtype: int64

In [168]:
pd.crosstab(index = isu_hist_schedule['game_year'], columns = isu_hist_schedule['game_result'])

game_result,L,N/A,T,W
game_year,,,,
2009,11,0,3,5
2010,12,0,1,8
2011,13,2,0,4
2012,9,2,3,9
2013,10,2,0,9
2014,9,0,3,5
2015,10,5,1,6
2016,11,2,2,7
2017,11,0,1,7


In [169]:
pd.crosstab(index = isu_hist_schedule['game_day_of_week'], columns = isu_hist_schedule['game_result'])

game_result,L,N/A,T,W
game_day_of_week,,,,
Fri,39,1,7,28
Mon,1,1,0,0
Sat,10,4,3,5
Sun,45,3,9,28
Thu,10,0,0,5
Tue,1,3,0,0
Wed,12,1,0,4
